In [ ]:
### Imports
# Modules
import csv
import cv2
import numpy as np

# Utilities
from imgaug import augmenters as iaa
from random import random
from sklearn.model_selection import train_test_split

# Keras
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D, Conv2D, Dropout
from keras.regularizers import l2

In [ ]:
# Constants
DATA_DIR = 'data/'
CSV_FILE = 'driving_log.csv'

# Generator parameters
BATCH_SIZE = 16

# Cropping parameters
TOP_STRIDE_TO_CROP = 30
BOTTOM_STRIDE_TO_CROP = 25

# Augmentation parameters
angle_treshold = 0.1

# Parameters
steering_correction = 0.25

In [ ]:
# Load csv log file
csv_file = []
with open(DATA_DIR+CSV_FILE) as csvfile:
    reader = csv.reader(csvfile)
    next(reader, None)  # skip the headers
    for line in reader:
        csv_file.append(line)

In [ ]:
# Split csv in train and validation sets
csv_train, csv_valid = train_test_split(csv_file, test_size = 0.3)

In [ ]:
# Image transformations
# Transformation list:
tr_list = [iaa.Add((-75, 75)), # change brightness of images (by -10 to 10 of original value)
           iaa.Multiply((0.5, 2.0)),
           iaa.ContrastNormalization((0.5, 2.0)), # improve or worsen the contrast
           iaa.Dropout((0.01, 0.1), per_channel=0.5), # randomly remove up to 10% of the pixels
           iaa.AdditiveGaussianNoise(scale=(0, 0.1*255))]
seq = iaa.SomeOf((0, 2), tr_list)

In [ ]:
def transformed_data_generator(csv_file, batch_size = BATCH_SIZE):

    # Camera parameters
    cameras = ['center', 'left', 'right']
    cameras_index = {'center':0, 'left':1, 'right':2} # 0:center, 1:left, 2:right
    cameras_steering_corretion = {'center':0, 'left':steering_correction, 'right':-steering_correction}
    
    # Bath definitions
    batch_train = np.zeros((batch_size, 160, 320, 3), dtype = np.uint8)
    batch_angle = np.zeros((batch_size,), dtype = np.float32)
    
    # Loop to generate data
    while True:
        
        # Generate data poins
        for idx in range(batch_size):
            
            # Get random csv line
            while True:
                
                line_idx = np.random.choice(len(csv_file))
                angle = float(csv_file[line_idx][3])
                # If angle above treshold get it
                if angle > angle_treshold:
                    break
                # if below, discard with 0.5 prob
                elif random() > 0.5:
                    break   
            
            ### Randomly choose center, left or right image
            # Get random camera 0:center, 1:left, 2:right 
            camera = np.random.choice(cameras)
            # Load image
            path = DATA_DIR + csv_file[line_idx][cameras_index[camera]].strip()
            batch_train[idx] = cv2.imread(path)
            # Adjust angle
            batch_angle[idx] = angle + cameras_steering_corretion[camera]
            
            ### Randomly flip
            # If flip image with 0.5 prob
            if random() > 0.5:
                batch_train[idx] = cv2.flip(batch_train[idx],1)
                batch_angle[idx] *= -1
        
        ### Random image transformation
        # up to two of the following transformations:
        # Bright augmentation
        # Multiply
        # Contrast normalization
        # Dropout ()
        # Additive Gaussian Noise
        batch_train = seq.augment_images(batch_train)
            
        yield batch_train, batch_angle

In [ ]:
def original_data_generator(csv_file, batch_size = BATCH_SIZE):
    
    batch_train = np.zeros((batch_size, 160, 320, 3), dtype = np.uint8)
    batch_angle = np.zeros((batch_size,), dtype = np.float32)
    while True:
        for idx in range(batch_size):
            # Get random csv line
            line_idx = np.random.choice(len(csv_file))
            # Load image 
            path = DATA_DIR + csv_file[line_idx][0].strip()
            batch_train[idx] = cv2.imread(path)
            # Load angle
            batch_angle[idx] = float(csv_file[line_idx][3])          
            
        yield batch_train, batch_angle

In [ ]:
train_data_generator = transformed_data_generator(csv_train)
valid_data_generator = original_data_generator(csv_valid)

In [ ]:
# Define simple model
model = Sequential()
model.add(Cropping2D(cropping=((30,25),(0,0)), input_shape = (160, 320, 3)))
model.add(Lambda(lambda x: x / 255.0 - 0.5))
model.add(Conv2D(24, (5, 5), padding = 'valid', strides = (2, 2),
                 kernel_initializer = 'random_uniform', bias_initializer = 'zeros',
                 kernel_regularizer = l2(0.001), bias_regularizer = l2(0.001),
                 activation = 'relu'))
model.add(Conv2D(36, (5, 5), padding = 'valid', strides = (2, 2),
                 kernel_initializer = 'random_uniform', bias_initializer = 'zeros',
                 kernel_regularizer = l2(0.001), bias_regularizer = l2(0.001),
                 activation = 'relu'))
model.add(Conv2D(48, (5, 5), padding = 'valid', strides = (2, 2),
                 kernel_initializer = 'random_uniform', bias_initializer = 'zeros',
                 kernel_regularizer = l2(0.001), bias_regularizer = l2(0.001),
                 activation = 'relu'))
model.add(Conv2D(64, (3, 3), padding = 'same', strides = (2, 2),
                 kernel_initializer = 'random_uniform', bias_initializer = 'zeros',
                 kernel_regularizer = l2(0.001), bias_regularizer = l2(0.001),
                 activation = 'relu'))
model.add(Conv2D(64, (3, 3), padding = 'valid', strides = (2, 2),
                 kernel_initializer = 'random_uniform', bias_initializer = 'zeros',
                 kernel_regularizer = l2(0.001), bias_regularizer = l2(0.001),
                 activation = 'relu'))
model.add(Flatten())
model.add(Dense(80, kernel_regularizer = l2(0.001), bias_regularizer = l2(0.001)))
model.add(Dropout(0.5))
model.add(Dense(40, kernel_regularizer = l2(0.001), bias_regularizer = l2(0.001)))
model.add(Dropout(0.5))
model.add(Dense(16, kernel_regularizer = l2(0.001), bias_regularizer = l2(0.001)))
model.add(Dropout(0.5))
model.add(Dense(10, kernel_regularizer = l2(0.001), bias_regularizer = l2(0.001)))
model.add(Dense(1, kernel_regularizer = l2(0.001), bias_regularizer = l2(0.001)))
model.summary()

In [ ]:
model.compile(loss = 'mse', optimizer = 'adam')
model.fit_generator(train_data_generator, steps_per_epoch = len(csv_train)/BATCH_SIZE,
                    nb_epoch=5, #FLAGS.epochs,
                    validation_data = valid_data_generator, validation_steps = len(csv_valid)/BATCH_SIZE)

In [ ]:
# Save model
model.save('model.h5')